# Segundo punto

Juan Sebastian Espitia

Anamaria Irmgard Mojica 201316580

María José Ocampo 201719100

Leidy Romero Santana 201714832

### Enunciado:
Comparación de estrategias de motores de búsqueda

A continuación, implementará un motor de búsqueda con cuatro estrategias diferentes.
1. Búsqueda binaria (BS).
2. Búsqueda binaria con índice invertido (BSII)
3. Recuperación clasificada básica (RRI)
4. Recuperación clasificada y vectorización de documentos (RRDV)
Debe hacer su propia implementación utilizando numpy y pandas para manejar matrices y matrices.

Nota: Hay puntos adicionales [15p] si su implementación de "índice invertido" está distribuida (usando para
ejemplo MapReduce) o la clasificación de discos eficiente se realiza mediante BSBI. Ambas estrategias se explican en
capítulo 4 del libro https://nlp.stanford.edu/IR-book/pdf/04const.pdf.
Conjunto de datos: hay tres archivos que componen el conjunto de datos. "Docs raws texts" contiene 331 documentos en
formato NAF (XML: debe utilizar el título y el contenido para modelar cada documento). "Queries raw text" contiene
35 consultas. "relevance-judgments.tsv" contiene para cada consulta los documentos relevantes. Estos documentos relevantes fueron construidos manualmente por jueces humanos y sirven como conjunto de datos de evaluación.

Pasos de preprocesamiento: para los siguientes puntos, debe preprocesar documentos y consultas utilizando tokenización por palabra, eliminación de stop words, normalización y stemming. 

Primero se instalan los paquetes necesarios


In [1]:
!pip install pandas
!pip install nltk
!pip install scikit-learn
!pip install gensim
!pip install spacy
!pip install textblob
!pip install pyspark

In [2]:
import pandas as pd
import numpy as np
import math
import re
import csv
from zipfile import ZipFile
from urllib.request import urlopen

# CARGA DE DATOS

In [3]:
docs=pd.read_csv('docs.csv', sep = ';', names=['ID','lemmas']).set_index('ID')
queries=pd.read_csv('queries.csv', sep = ';', names=['ID','lemmas']).set_index('ID')
lemmas=open("diccionario-lemmas-nltk.txt", "r")
N=len(docs)
len(queries)

35

In [4]:
# Creacion del indice para los documentos
indice_invertido = {} 
f = open("diccionario-lemmas-nltk.txt", "r")
for lemma in f:
  indice_invertido[lemma.replace('\n', '')]=[]

for index, row in docs.iterrows():
  lemmas=row['lemmas'].split(',')
  for lemma in lemmas:
    indice_invertido[lemma].append(index)

print(indice_invertido) 

{'theta': ['d307'], 'seminary': ['d263', 'd314'], 'scout': ['d089', 'd158', 'd158', 'd158', 'd280'], 'plumeria': ['d317', 'd317'], 'derives': ['d215'], 'unquestionable': ['d179'], 'admission': ['d039', 'd039', 'd170', 'd263'], 'action': ['d007', 'd021', 'd037', 'd072', 'd113', 'd130', 'd131', 'd134', 'd139', 'd139', 'd139', 'd158', 'd171', 'd202', 'd233', 'd254', 'd254', 'd263', 'd272', 'd286', 'd303'], 'repeat': ['d111'], 'moor': ['d228', 'd228', 'd316', 'd318', 'd318'], 'verrier': ['d140', 'd140', 'd140', 'd140', 'd140', 'd140', 'd140', 'd263'], 'gerhard': ['d058'], 'utah': ['d071', 'd138', 'd199', 'd302', 'd302', 'd312'], 'gogh': ['d109'], 'moses': ['d172'], 'stehen': ['d326'], 'lynch': ['d084'], 'ex': ['d078', 'd078', 'd160', 'd234'], 'logic': ['d005', 'd006', 'd083', 'd092', 'd092', 'd092', 'd193', 'd197', 'd202', 'd207', 'd211', 'd212', 'd212', 'd212', 'd212', 'd212', 'd212', 'd212', 'd214', 'd214', 'd215', 'd215', 'd220', 'd220', 'd220', 'd220', 'd221', 'd221', 'd224', 'd227', '

In [5]:
# Creacion del indice para las queries
indice_invertido_queries = {} 
for index, row in queries.iterrows():
  lemmas=row['lemmas'].split(',')
  for lemma in lemmas:
    if lemma in indice_invertido_queries:
        indice_invertido_queries[lemma].append(index)
    else:
        indice_invertido_queries[lemma]=[index]

print(indice_invertido_queries) 

{'fabrication': ['q01'], 'music': ['q01'], 'instrument': ['q01'], 'famous': ['q02', 'q13', 'q23'], 'german': ['q02'], 'poetry': ['q02'], 'romanticism': ['q03'], 'university': ['q04'], 'edinburgh': ['q04'], 'research': ['q04'], 'bridge': ['q06'], 'construction': ['q06'], 'walk': ['q07'], 'fame': ['q07'], 'star': ['q07'], 'scientist': ['q08', 'q34', 'q38'], 'worked': ['q08'], 'atomic': ['q08'], 'bomb': ['q08'], 'invention': ['q09', 'q14'], 'internet': ['q09'], 'early': ['q10'], 'telecommunication': ['q10'], 'method': ['q10'], 'explored': ['q12'], 'south': ['q12', 'q16'], 'pole': ['q12'], 'member': ['q13'], 'royal': ['q13'], 'navy': ['q13'], 'nobel': ['q14', 'q46'], 'prize': ['q14', 'q46'], 'winning': ['q14'], 'america': ['q16'], 'edward': ['q17'], 'teller': ['q17'], 'marie': ['q17'], 'curie': ['q17'], 'computing': ['q18'], 'language': ['q18'], 'programming': ['q18'], 'artificial': ['q18'], 'intelligence': ['q18'], 'william': ['q19'], 'hearst': ['q19'], 'movie': ['q19'], 'captain': ['q22'

# RECUPERACIÓN RANQUEADA Y VECTORIZACION DE DOCUMENTOS (RRDV)
1. Cree una función que, a partir del índice invertido, cree un vector ponderado tf.idf que represente un documento o un query. 
Describa su estrategia detalladamente, ¿Es eficiente? ¿Por qué?
2. Cree una función que reciba dos vectores de documentos y compute la similaridad de coseno.
3. Para cada una de las 35 queries del dataset, obtenga los documentos rankeados -ordenando por el puntaje de similaridad de coseno- (Incluya solo los documentos con un puntaje superior a 0 para una query dada). Escriba en un archivo (RRDV-queries_results) los resultados siguiendo el mismo formato que en el archivo "relevance-judgments": 
q01 dXX: cos_simi(q01,dXX),dYY: cos_simi(q01, dYY),dZZ: cos_simi(q01,dZZ)...
4. Calcule P@M, R@M, NDCG@M por consulta. M es el número de documentos relevantes que se encuentran en el archivo de relevance-judgments por consulta. Luego calcule MAP como métrica general.

NOTA I: Para P@M y R@M, suponga una escala de relevancia binaria. Los documentos que no se encuentran en el archivo de relevance-judgments NO son relevantes para una consulta determinada.

NOTA II: Para NDCG@M use la escala de relevancia no binaria que se encuentra en el archivo de relevance-judgments.

In [6]:
#1. Se define la funcion
def ii_to_tdidf(index, docID):
    tdidf = {}
    for key in index.keys():
        if docID in index[key]:
            count=np.count_nonzero(np.array(index[key]) == docID)
            docs = np.unique(index[key])
            tdidf[key] = np.log10(N/count)*np.log(1+len(docs))
    return tdidf

In [7]:
def vector_dicts_to_arrays(vector_dict1, vector_dict2):
    total_keys=np.unique(np.concatenate((np.array(list(vector_dict1.keys())),np.array(list(vector_dict2.keys())))))
    arr1 = []
    arr2 = []
    for k in total_keys:
        arr1.append(vector_dict1[k] if k in vector_dict1 else 0)
        arr2.append(vector_dict2[k] if k in vector_dict2 else 0)
    return np.array(arr1), np.array(arr2) 

def cosine_similarity(doc_a, doc_b):
    return np.dot(doc_a, doc_b) / (np.linalg.norm(doc_a) * np.linalg.norm(doc_b))

In [8]:
results={}
for indexQ, rowQ in queries.iterrows():
    vector_query=ii_to_tdidf(indice_invertido_queries, indexQ)
    for indexD, rowD in docs.iterrows():
        print(indexQ, indexD)
        vector_doc=ii_to_tdidf(indice_invertido, indexD)
        q, d = vector_dicts_to_arrays(vector_query, vector_doc)
        puntuacion=cosine_similarity(q, d)
        if puntuacion>0:
            if indexQ in results.keys():
                results[indexQ].append((indexD,round(puntuacion,4)))
            else:                
                results[indexQ]=[]
print(results)
for q in results.values():
    q.sort(key = lambda x: x[1], reverse = True)


q01 d001
q01 d002
q01 d003
q01 d004
q01 d005
q01 d006
q01 d007
q01 d008
q01 d009
q01 d010
q01 d011
q01 d012
q01 d013
q01 d014
q01 d015
q01 d016
q01 d017
q01 d018
q01 d019
q01 d020
q01 d021
q01 d022
q01 d023
q01 d024
q01 d025
q01 d026
q01 d027
q01 d028
q01 d029
q01 d030
q01 d031
q01 d032
q01 d033
q01 d034
q01 d035
q01 d036
q01 d037
q01 d038
q01 d039
q01 d040
q01 d041
q01 d042
q01 d043
q01 d044
q01 d045
q01 d046
q01 d047
q01 d048
q01 d049
q01 d050
q01 d051
q01 d052
q01 d053
q01 d054
q01 d055
q01 d056
q01 d057
q01 d058
q01 d059
q01 d060
q01 d061
q01 d062
q01 d063
q01 d064
q01 d065
q01 d066
q01 d067
q01 d068
q01 d069
q01 d070
q01 d071
q01 d072
q01 d073
q01 d074
q01 d075
q01 d076
q01 d077
q01 d078
q01 d079
q01 d080
q01 d081
q01 d082
q01 d083
q01 d084
q01 d085
q01 d086
q01 d087
q01 d088
q01 d089
q01 d090
q01 d091
q01 d092
q01 d093
q01 d094
q01 d095
q01 d096
q01 d097
q01 d098
q01 d099
q01 d100
q01 d101
q01 d102
q01 d103
q01 d104
q01 d105
q01 d106
q01 d107
q01 d108
q01 d109
q01 d110
q01 d111
q

q03 d264
q03 d265
q03 d266
q03 d267
q03 d268
q03 d269
q03 d270
q03 d271
q03 d272
q03 d273
q03 d274
q03 d275
q03 d276
q03 d277
q03 d278
q03 d279
q03 d280
q03 d281
q03 d282
q03 d283
q03 d284
q03 d285
q03 d286
q03 d287
q03 d288
q03 d289
q03 d290
q03 d291
q03 d292
q03 d293
q03 d294
q03 d295
q03 d296
q03 d297
q03 d298
q03 d299
q03 d300
q03 d301
q03 d302
q03 d303
q03 d304
q03 d305
q03 d306
q03 d307
q03 d308
q03 d309
q03 d310
q03 d311
q03 d312
q03 d313
q03 d314
q03 d315
q03 d316
q03 d317
q03 d318
q03 d319
q03 d320
q03 d321
q03 d322
q03 d323
q03 d324
q03 d325
q03 d326
q03 d327
q03 d328
q03 d329
q03 d330
q03 d331
q04 d001
q04 d002
q04 d003
q04 d004
q04 d005
q04 d006
q04 d007
q04 d008
q04 d009
q04 d010
q04 d011
q04 d012
q04 d013
q04 d014
q04 d015
q04 d016
q04 d017
q04 d018
q04 d019
q04 d020
q04 d021
q04 d022
q04 d023
q04 d024
q04 d025
q04 d026
q04 d027
q04 d028
q04 d029
q04 d030
q04 d031
q04 d032
q04 d033
q04 d034
q04 d035
q04 d036
q04 d037
q04 d038
q04 d039
q04 d040
q04 d041
q04 d042
q04 d043
q

KeyboardInterrupt: 

In [9]:
# Se escribe el archivo RRI-queries_results
with open('RRDV-queries_results.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for item in results.items():
      tsv_writer.writerow([item[0], re.sub(r'[\[\]\(\)\ ]', '', str(item[1]).replace("',",":").replace("'",""))])

In [14]:
# Se calculan las metricas de evaluacion para el conjunto obtenido

# Lectura de relevance-judgments
request_url_3 = urlopen('https://github.com/LeidyRomero/data_nlp/raw/main/HW01/relevance-judgments.tsv') 
z3 = request_url_3.read()

f = open("relevance-judgments.tsv", "wb")
f.write(z3)
f.close()

tsv = open("relevance-judgments.tsv","r")
read_tsv = csv.reader(tsv, delimiter="\t")

sum_avg_precision = 0
Q = 46

for row in read_tsv:
    try:
        #query
        print('-'*50)
        print('-'*23+row[0]+'-'*23)
        print('-'*50)
        #docs rel
        #print(row[1])
        M = len(row[1].split(","))
        #docs rel
        rel = {}
        for r in row[1].split(","):
            rel[r.split(":")[0]] = r.split(":")[1]
        rel_docs=list(rel.keys())

        #docs ret
        ret = [i[0] for i in results.get(row[0])[:M]]

        DCG_ACUM = 0
        BEST_DCG_ACUM = 0

        #BEST DCG
        BEST_DCG = list(rel.values())
        BEST_DCG.sort(reverse = True)
        #print(BEST_DCG)

        sum_precision = 0
        cont_precision = 0
        numerador = None
        antiguo_numerador = None

        for i in range(0,M):

            if numerador is not None:
                antiguo_numerador = numerador

            numerador = [value for value in rel_docs[:i+1] if value in ret[:i+1]] 

            if antiguo_numerador is not None and len(antiguo_numerador)!=len(numerador):
                sum_precision += len(numerador)/(i+1)
                print('entra')

            print('Rel = '+str(rel_docs[:i+1])+', Ret = '+str(ret[:i+1]))
            print('P@'+str(i+1)+': '+str(len(numerador)/(i+1)))
            print('R@'+str(i+1)+': '+str(len(numerador)/M))

            discount_factor = 1/(np.log2(np.max(np.array([i+1,2]))))
            #DCG 
            rel_i = rel.get(ret[i])
            if rel_i is not None:
                DCG_ACUM += discount_factor*int(rel_i)

            #BEST_DCG_2
            BEST_DCG_ACUM += discount_factor*int(BEST_DCG[i])

            print('NDCG@'+str(i+1)+': '+str(DCG_ACUM/BEST_DCG_ACUM))

        print('-'*20)
        sum_avg_precision += sum_precision/M
    except Exception as e:
        print(e)
        continue
print('MAP: '+ str(sum_avg_precision/Q))
tsv.close()

--------------------------------------------------
-----------------------q01-----------------------
--------------------------------------------------
Rel = ['d186'], Ret = ['d006']
P@1: 0.0
R@1: 0.0
NDCG@1: 0.0
Rel = ['d186', 'd254'], Ret = ['d006', 'd016']
P@2: 0.0
R@2: 0.0
NDCG@2: 0.5
entra
Rel = ['d186', 'd254', 'd016'], Ret = ['d006', 'd016', 'd028']
P@3: 0.3333333333333333
R@3: 0.3333333333333333
NDCG@3: 0.3992424290497487
--------------------
--------------------------------------------------
-----------------------q02-----------------------
--------------------------------------------------
Rel = ['d136'], Ret = ['d002']
P@1: 0.0
R@1: 0.0
NDCG@1: 0.0
Rel = ['d136', 'd139'], Ret = ['d002', 'd003']
P@2: 0.0
R@2: 0.0
NDCG@2: 0.0
Rel = ['d136', 'd139', 'd143'], Ret = ['d002', 'd003', 'd004']
P@3: 0.0
R@3: 0.0
NDCG@3: 0.0
Rel = ['d136', 'd139', 'd143', 'd283'], Ret = ['d002', 'd003', 'd004', 'd005']
P@4: 0.0
R@4: 0.0
NDCG@4: 0.0
Rel = ['d136', 'd139', 'd143', 'd283', 'd228'], Ret =